# On dynamic range: 'roundoff errors' and the FFT

### Background
Accuracy of the FFT has been a consideration since (at least) the initial implementation of the Cooley-Tukey algorithm, there are a handful papers on the subject going right back to the 60's - 'accuracy' and 'round-off error' being the most relevant search terms. The best recent references I could find are:

* [Accuracy of the Discrete Fourier Transform and the Fast Fourier Transform (Schatzman 1996)](https://doi.org/10.1137/S1064827593247023) - a relatively recent review of the theory

* [Implementing FFTs in Practice (Johnson et al. 2008)](http://cnx.org/content/m16336/) - from the authors of FFTW

The consensus seems to be that the relative error from a (carefully implemented and tuned) FFT of input length N grows as $O(\log{N})$ in the worst case, and $O(\sqrt{\log{N}})$ on average for the more amenable case of random input data. This applies to one-dimensional FFT's, but (perhaps surprisingly) the error-growth curve for a 2-dimensional FFT is only slightly worse and of the same order (cf $\S6$, Schatzman 1996).


However, it's worth noting that the typical results (as reported in the [FFTW accuracy benchmarks](http://www.fftw.org/accuracy/comments.html)) are obtained by testing with random data (computed with arbitrarily high precision and then compare with results computed via standard single / double floating precision). So when considering a radio-field with a bright source and a faint source, we may be in the 'worst-case' regime.

This is a typical benchmark plot for FFTW:
![Illustrative FFTW benchmark plot](http://www.fftw.org/accuracy/Ryzen-7-3.6GHz/ryzen.1d.scxx.acc.p2.png)

This has a similar shape to the best-case curves from Schatzman 1996 (cf $\S3.4$).
For an FFT of length $N$ we expect a relative error in the random-noise 'typical' cose of order $$\sqrt{\log{N}}\epsilon = 3\epsilon$$
where $\epsilon$ is the machine accuracy --- recall this is  $\frac{1}{2^{24}}$ in the single-precision case.
So we can run a quick calculation, generate our own plot, and check if the FFTW plot agrees with our understanding of the theory:

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (12,8)
import numpy as np
from matplotlib.ticker import LogLocator


N = np.asarray([float(2**i) for i in range(4,18)])
epsilon = 1./ 2**24
arbitrary_scaling_factor = 0.6 # cf Schatzman 1996, error function 3b

def fft_err(N):
    return arbitrary_scaling_factor* np.sqrt(np.log2(N))  * epsilon

plt.plot(N, fft_err(N))
ax = plt.gca()
ax.set_yscale('log')
ax.set_xscale('log', basex=2)

# plt.xticks(N)
plt.xlim(min(N), max(N))
plt.ylim(0.5e-7, 1.5e-7)
plt.axhline(1e-7, ls=':')
plt.xlabel('N')
plt.ylabel('Relative error')

Result - not a perfect agreement, but in the right ballpark.